# User's choice

In [6]:
# for all the paths, do not add the final slash

project_path = ""  # you should keep everything in the same main folder

# where the images are : example --> + "/data/FDV_cell_database"
images_path = project_path + ""

# where the results go
# for centroid csv : example --> + "/how_are_cells_connected/junctions_centroids_database"
centroid_result_path = project_path + ""

# for masks : example --> + "/intermediate_results/junction_masks"
mask_result_path = project_path + ""


## Full example

### enter the path to the image you want to analyze 
images_path = "/Users/camille/Documents/project plants/data/FDV_cell_database"

### enter the path to the place you want to store the masks once you have applied them to the images
mask_result_path = "/Users/camille/Documents/project plants/intermediate_results/junction_masks"

### enter the path to the place you want to store the multiple csv with the centroids 
centroid_result_path = "/Users/camille/Documents/project plants/how_are_cells_connected/junctions_centroids_database"

# All imports


In [5]:
#Import all necessary packages

from scipy import ndimage as ndi
import pandas as pd
import napari
import numpy as np
import tifffile
import skimage
import glob
import matplotlib.pyplot as plt

from tifffile import imread, imwrite
from skimage import morphology
from skimage.filters import threshold_local
from skimage import data          
from skimage.morphology import disk
from skimage.morphology import square
from skimage.filters import median 
from skimage import color
from skimage.morphology import skeletonize 
from skimage.measure import label, regionprops, regionprops_table
from sklearn.neighbors import NearestNeighbors


# import from the .py file associated to this code
from src.junctions_functions import segmentation_junctions, total_junctions, non_triangular_junctions, nbr_junctions_total_csv

ImportError: cannot import name 'total_junctions' from 'src.junctions_functions' (/Users/camille/Documents/project plants/src/junctions_functions.py)

# 1. create parameters for the 6 segmentations

In [2]:
# list of the parameter of the segmentation depending on the cells type in the image
# the name of the folders in which we store each type should be as following in red 

parameters = [
    # [disk_size, block_size, min_size_object, erosion_size]
    
    ["/Half shape", 8, 111, 9500, 2], 
    ["/Croco shape", 8, 121, 10500, 2], 
    ["/Long shape", 8, 121, 8500, 2], 
    ["/Puzzle shape", 8, 121, 10000, 3], 
    ["/Round shape", 8, 97, 10000, 2], 
    ["/5_20 shape", 12, 171, 10000, 3], # noyaux très foncés donc un threshold plus élevé et plus d'érosion
]

# 2. create masks for all the image

In [ ]:
# this function creates masks for all images by using the parameters we determined before

for param in parameters:
    
    # runs on all the paths to each image
    for image_path in glob.glob(images_path + param[0] + "/*.tif"): 
        
        print(image_path)
        
        # applies the segmentation on all the images with their appropriate parameters
        skel = segmentation_junctions(image_path, *param[1:])
        
        # spilts the path to the image by the sign "/"
        image_split = image_path.split("/")
        # recover the image name by keeping the last element
        image_name = image_split[-1]
        
        # stores the mask as a tifffile file in the place indicated by the "mask_result_path"
        tifffile.imwrite(mask_result_path + "/" + image_name, skel)

# 3. Create final csv:
## image name, number of non-triangular junctions, total number of junctions, centroids of non-tringular junctions


In [ ]:
# executes the function: create the final csv with results 

final_df = nbr_junctions_total_csv(images_path, mask_result_path, centroid_result_path)

# 4. Create bar graph

In [ ]:
# adds the code for each specie to the final cvs, it corresponds to the begining of the name of each image
final_df["species code"] = final_df['image name'].str.split('-').str[:2].str.join("-")
final_df

In [ ]:
# creates a dictionary that links each specie's code to the name of the specie
map = {
    "3-07" : "Ceratostigma_minus",
    "3-15" : "Rosa_mollis",
    "4-01": "Persicaria_virginiana",
    "4-02": "Polygonum_chinense",
    "4-15": "Paeonia_tenuifolia",
    "4-17": "Aconitum_carmichaelii",
    "5-20": "Succisella_inflexa",
    "5-22": "Beta_vulgaris",
    "6-19": "Galium_rubioides",
    "6-21": "Galium_odoratum",
    "7-01": "Danae_racemosa",
    "7-15":"Rohdea_japonica",
    "7-23": "Alstromemeria_aurea",
    "7-24": "Hemerocallis_fulva",
    "8-06": "Jasminum_fruticans"}

In [ ]:
# replaces the species code by the name of the species
final_df = final_df.replace({"species code": map})
final_df

In [ ]:
# create bar plot with seaborn

import seaborn as sns

final_df["number non-triangular junctions"] = pd.to_numeric(final_df["number non-triangular junctions"])
final_df = final_df.assign(junctions_ratio = final_df["number non-triangular junctions"]/final_df["total nb of junctions"])

                           
sns.barplot(data=final_df, y="species code", x="junctions_ratio")
plt.show()